In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
from google.cloud import bigquery
client = bigquery.Client()
query = "SELECT * FROM `customer-churn-451414.customer_churn_data.customer_churn`"
df = client.query(query).to_dataframe()


In [ ]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,...,InternetService_no,Contract_month-to-month,Contract_one_year,Contract_two_year,PaymentMethod_electronic_check,PaymentMethod_mailed_check,PaymentMethod_bank_transfer__automatic_,PaymentMethod_credit_card__automatic_,AvgMonthlySpend,TenureGroup
0,3115-CZMZD,Male,0,0,1,0,1,No,No internet service,No internet service,...,1,<NA>,<NA>,1,<NA>,1,<NA>,<NA>,0.0,New
1,7644-OMVMY,Male,0,1,1,0,1,No,No internet service,No internet service,...,1,<NA>,<NA>,1,<NA>,1,<NA>,<NA>,0.0,New
2,2520-SGTTA,Female,0,1,1,0,1,No,No internet service,No internet service,...,1,<NA>,<NA>,1,<NA>,1,<NA>,<NA>,0.0,New
3,2923-ARZLG,Male,0,1,1,0,1,No,No internet service,No internet service,...,1,<NA>,1,<NA>,<NA>,1,<NA>,<NA>,0.0,New
4,3213-VVOLG,Male,0,1,1,0,1,Yes,No internet service,No internet service,...,1,<NA>,<NA>,1,<NA>,1,<NA>,<NA>,0.0,New


In [ ]:
print(df.columns.tolist())

['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'MonthlyCharges', 'TotalCharges', 'Churn', 'InternetService_dsl', 'InternetService_fiber_optic', 'InternetService_no', 'Contract_month-to-month', 'Contract_one_year', 'Contract_two_year', 'PaymentMethod_electronic_check', 'PaymentMethod_mailed_check', 'PaymentMethod_bank_transfer__automatic_', 'PaymentMethod_credit_card__automatic_', 'AvgMonthlySpend', 'TenureGroup']


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 30 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   customerID                               7043 non-null   object 
 1   gender                                   7043 non-null   object 
 2   SeniorCitizen                            7043 non-null   Int64  
 3   Partner                                  7043 non-null   Int64  
 4   Dependents                               7043 non-null   Int64  
 5   tenure                                   7043 non-null   Int64  
 6   PhoneService                             7043 non-null   Int64  
 7   MultipleLines                            7043 non-null   object 
 8   OnlineSecurity                           7043 non-null   object 
 9   OnlineBackup                             7043 non-null   object 
 10  DeviceProtection                         7043 no

In [ ]:
X = df.drop(columns=["customerID", "Churn"])
y = df["Churn"]


In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Load data (assuming you have already fetched it from BigQuery)
df = pd.DataFrame(X)  # X is your input data

# Identify numerical and categorical columns
numerical_cols = df.select_dtypes(include=[np.number]).columns
categorical_cols = df.select_dtypes(exclude=[np.number]).columns

# One-hot encode categorical columns
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Standardize numerical features
scaler = StandardScaler()
df_encoded[numerical_cols] = scaler.fit_transform(df_encoded[numerical_cols])

# Convert back to NumPy if needed
X_scaled = df_encoded.values

print("Data successfully scaled and encoded!")


Data successfully scaled and encoded!


In [ ]:
print(X_scaled)

[[-0.43991649313097614 -0.9666223112813243 1.5290240978948397 ... False
  False True]
 [-0.43991649313097614 1.0345302279174904 1.5290240978948397 ... False
  False True]
 [-0.43991649313097614 1.0345302279174904 1.5290240978948397 ... False
  False True]
 ...
 [-0.43991649313097614 -0.9666223112813243 -0.6540119291623984 ... True
  False False]
 [-0.43991649313097614 1.0345302279174904 1.5290240978948397 ... True
  False False]
 [-0.43991649313097614 1.0345302279174904 1.5290240978948397 ... True
  False False]]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [ ]:
# Build Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
import numpy as np
import tensorflow as tf

# Ensure all features are float32
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

# Ensure labels are int32 if they are categorical (for classification)
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

# Train Model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


Epoch 1/10
177/177 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7290 - loss: nan - val_accuracy: 0.7317 - val_loss: nan
Epoch 2/10
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7267 - loss: nan - val_accuracy: 0.7317 - val_loss: nan
Epoch 3/10
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7485 - loss: nan - val_accuracy: 0.7317 - val_loss: nan
Epoch 4/10
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7452 - loss: nan - val_accuracy: 0.7317 - val_loss: nan
Epoch 5/10
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7395 - loss: nan - val_accuracy: 0.7317 - val_loss: nan
Epoch 6/10
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7297 - loss: nan - val_accuracy: 0.7317 - val_loss: nan
Epoch 7/10
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7319 - loss: nan - val_accuracy: 0.7317 - val_loss: nan
Epoch 8/10
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7434 - loss: nan - val_accuracy: 0.7317 - val_loss: nan
Epoch 9/10
177/177 ━━━━━

In [ ]:
# Save in Keras format (Recommended)
model.save("churn_model.keras")




In [ ]:
!gsutil cp -r churn_model.keras gs://customer-churn-storage-bucket/models/

Copying file://churn_model.keras [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/29.0 KiB.                                     


In [ ]:
from google.colab import auth
auth.authenticate_user()

!gcloud auth login --quiet
!gcloud config set project customer-churn-451414


Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=Ky2ZGpTPvOZtbl2ti1Ld4WF7ZvcuUA&prompt=consent&token_usage=remote&access_type=offline&code_challenge=VDU4jlZDTfD-qX6p3YoCAiGikRZg-iNMbGUd5IfmINo&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AQSTgQEq9t-_B6UoWl5D0E9opsa57X7o4KMUM_hOlqPvMHmy-lVDg42l8bdtAeYnSf29vQ

You are now logged in as [farisayob98@gmail.com].
Your current project 

In [ ]:
!gcloud config get-value ai/region


(unset)


In [ ]:
!gcloud config set ai/region us-central1


Updated property [ai/region].


In [ ]:
!gcloud config get-value ai/region

us-central1


In [ ]:
!gcloud ai models list --region=us-central1


Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Listed 0 items.


In [ ]:
!gsutil ls gs://customer-churn-storage-bucket/models/


gs://customer-churn-storage-bucket/models/
gs://customer-churn-storage-bucket/models/churn_model.keras


In [ ]:
!gsutil ls gs://customer-churn-storage-bucket/models/


gs://customer-churn-storage-bucket/models/
gs://customer-churn-storage-bucket/models/churn_model.keras


In [ ]:
!gcloud storage buckets list


---
creation_time: 2025-02-19T14:03:29+0000
default_storage_class: STANDARD
generation: 1739973807725754051
location: ASIA
location_type: multi-region
metageneration: 2
name: customer-churn-storage-bucket
public_access_prevention: inherited
rpo: DEFAULT
soft_delete_policy:
  retentionDurationSeconds: '0'
storage_url: gs://customer-churn-storage-bucket/
uniform_bucket_level_access: true
update_time: 2025-02-25T14:19:41+0000


In [ ]:
!gsutil ls gs://customer-churn-storage-bucket/models/


gs://customer-churn-storage-bucket/models/
gs://customer-churn-storage-bucket/models/churn_model.keras


In [ ]:
# Specify a directory to save the model
save_path = 'gs://customer-churn-storage-bucket/models/'

# Save the model in SavedModel format
tf.saved_model.save(model, save_path)

In [ ]:
!gcloud ai models upload \
    --region=asia-southeast1 \
    --display-name=saved_model.pb \
    --artifact-uri=gs://customer-churn-storage-bucket/models/ \
    --container-image-uri=gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-8:latest



Using endpoint [https://asia-southeast1-aiplatform.googleapis.com/]


In [ ]:
!gsutil ls gs://customer-churn-storage-bucket/models/


gs://customer-churn-storage-bucket/models/
gs://customer-churn-storage-bucket/models/churn_model.keras
gs://customer-churn-storage-bucket/models/fingerprint.pb
gs://customer-churn-storage-bucket/models/saved_model.pb
gs://customer-churn-storage-bucket/models/assets/
gs://customer-churn-storage-bucket/models/variables/


In [ ]:
!gcloud ai models list --region=asia-southeast1

Using endpoint [https://asia-southeast1-aiplatform.googleapis.com/]
MODEL_ID             DISPLAY_NAME
6478150987042258944  saved_model.pb


In [ ]:
!gcloud ai endpoints create \
    --region=asia-southeast1 \
    --display-name=churn_model_endpoint


Using endpoint [https://asia-southeast1-aiplatform.googleapis.com/]
Created Vertex AI endpoint: projects/236600548056/locations/asia-southeast1/endpoints/6825895728582230016.


In [ ]:
!gcloud version

Google Cloud SDK 505.0.0


In [ ]:
!gcloud ai endpoints list --region=asia-southeast1

Using endpoint [https://asia-southeast1-aiplatform.googleapis.com/]
ENDPOINT_ID          DISPLAY_NAME
6825895728582230016  churn_model_endpoint


In [ ]:
!gcloud ai endpoints list --region=asia-southeast1 --format="value(name)"

Using endpoint [https://asia-southeast1-aiplatform.googleapis.com/]
6825895728582230016


In [ ]:
!gcloud config set project 236600548056

Updated property [core/project].


In [ ]:
!gcloud services enable aiplatform.googleapis.com --project=236600548056

In [ ]:
!gcloud ai endpoints deploy-model 6825895728582230016 \
  --region=asia-southeast1 \
  --display-name=churn_model_deployment \
  --model=6478150987042258944 \
  --machine-type=n1-standard-4 \
  --traffic-split=0=100

Using endpoint [https://asia-southeast1-aiplatform.googleapis.com/]
Deployed a model to the endpoint 6825895728582230016. Id of the deployed model: 3139743414044590080.


In [ ]:
!gcloud projects list

PROJECT_ID                NAME               PROJECT_NUMBER
abiding-base-449307-d4    My First Project   403515473097
customer-churn-451414     customer-churn     236600548056
taxi-data-project-449307  taxi-data-project  804700731604


In [ ]:
!gcloud config set core/project customer-churn-451414  # Replace 'my-project-123' with your actual project ID

Updated property [core/project].


In [ ]:
!gcloud ai endpoints deploy-model --help

NAME
    gcloud ai endpoints deploy-model - deploy a model to an existing Vertex AI
        endpoint

SYNOPSIS
    gcloud ai endpoints deploy-model (ENDPOINT : --region=REGION)
        --display-name=DISPLAY_NAME --model=MODEL
        [--accelerator=[count=COUNT],[type=TYPE]]
        [--autoscaling-metric-specs=[METRIC-NAME=TARGET,...]]
        [--deployed-model-id=DEPLOYED_MODEL_ID] [--disable-container-logging]
        [--enable-access-logging] [--machine-type=MACHINE_TYPE]
        [--max-replica-count=MAX_REPLICA_COUNT]
        [--min-replica-count=MIN_REPLICA_COUNT]
        [--reservation-affinity=[key=KEY],
          [reservation-affinity-type=RESERVATION-AFFINITY-TYPE],
          [values=VALUES]] [--service-account=SERVICE_ACCOUNT] [--spot]
        [--traffic-split=[DEPLOYED_MODEL_ID=VALUE,...]] [GCLOUD_WIDE_FLAG ...]

EXAMPLES
    To deploy a model 456 to an endpoint 123 under project example in region
    us-central1, run:

        $ gcloud ai endpoints deploy-model 123 --proje